# EasyOCR Sample Application


# Prerequisites (DO NOT SKIP)

1. **PLEASE READ THE [README](README.md) INCLUDE WITH THIS BEFORE YOU START USING THIS NOTEBOOK**

# Set up

Import libraries for use with this notebook environment, you do not need these libraries when you write your application code.

In [ ]:
import sys
import os
import time
import json

import boto3
import sagemaker

import matplotlib.pyplot as plt
from IPython.core.magic import register_cell_magic

sys.path.insert( 0, os.path.abspath( "../common/test_utility" ) )
import panorama_test_utility

# instantiate boto3 clients
s3_client = boto3.client('s3')
panorama_client = boto3.client('panorama', region_name = 'us-west-2')

# configure matplotlib
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,20)

# register custom magic command
@register_cell_magic
def save_cell(line, cell):
    'Save python code block to a file'
    with open(line, 'wt') as fd:
        fd.write(cell)

## Notebook parameters
Global constants that help the notebook create Panorama resources on your behalf.

In [ ]:
# application name
app_name = 'lab4'

## package names and node names
code_package_name = 'lab4'
camera_node_name = 'abstract_rtsp_media_source'

# AWS account ID
account_id = boto3.client("sts").get_caller_identity()["Account"]

## Set up application

Every application uses the creator's AWS Account ID as the prefix to uniquely identifies the application resources. Running `panorama-cli import-application` replaces the generic account Id with your account Id.

In [ ]:
!cd ./lab4 && panorama-cli import-application

## Download Depedencies and Artifacts (One Time Download)

In [ ]:
panorama_test_utility.download_artifacts_gpu_sample('lab4', account_id)

## Update camera streams

In the AWS Panorama console, you can select the camera streams, but programmatically, you need to define the camera stream info for the cameras you are using with the app.

We used an ```abstract data source``` here, usually this lets you select the pre-created camera source from the console. But programatically, we have to do the following steps


- Create Camera
- Create Override json file
- Include the Override json file while are deploying the application

### Create New Camera

Because we are using an ```abstract_rtsp_media_source```, we have to create a camera before we can use the ```abstract_rtsp_media_source```

**NOTE** : Update your RTSP Info in the next cell, Username, Password and RTSP Stream URL

In [ ]:
CAMERA_NAME = "lab4_cam"

CAMERA_CREDS = '{"Username":"root","Password":"Aws2017!","StreamUrl": "rtsp://10.92.202.65/onvif-media/media.amp?profile=profile_1_h264&sessiontimeout=60&streamtype=unicast"}'

In [ ]:
res = !aws panorama create-node-from-template-job --template-type RTSP_CAMERA_STREAM \
    --output-package-name {CAMERA_NAME} \
    --output-package-version '3.0' \
    --node-name {CAMERA_NAME} \
    --template-parameters '{CAMERA_CREDS}'

# FIXME : camera node creation fails if it already exists.
# Should either ignore the already-exist error, or delete the node at the end of this notebook

res = ''.join(res)
print(res)
res_json = json.loads(res)

In [ ]:
!aws panorama describe-node-from-template-job --job-id {res_json['JobId']}

## Overriding camera node

If you want to override the camera configuration at deployment (for ex. deploy to another site) you can provide a deployment time override. Go to `people_counter_app/deployment_overrides/override_camera.json` file and replace YOUR_AWS_ACCOUNT_ID with your ACCOUNT_ID and YOUR_CAMERA_NAME with your camera name.

In [ ]:
# Update Account ID
with open( f"./{app_name}/deployment_overrides/override_camera.json", "r" ) as fd:
    override_json = json.load(fd)

override_json['nodeGraphOverrides']['packages'][0]['name'] = '{}::{}'.format(account_id, CAMERA_NAME)
override_json['nodeGraphOverrides']['nodes'][0]['name'] = CAMERA_NAME
override_json['nodeGraphOverrides']['nodes'][0]['interface'] = '{}::{}.{}'.format(account_id, CAMERA_NAME, CAMERA_NAME)    
override_json['nodeGraphOverrides']['nodeOverrides'][0]['with'][0]['name'] = CAMERA_NAME    

with open( f"./{app_name}/deployment_overrides/override_camera.json", "w") as fd:
    json.dump(override_json, fd)

### Upload application to Panorama for deploying to devices

In [ ]:
container_asset_name = 'lab4'

In [ ]:
# This step takes some time, depending on your network environment.

!cd ./lab4 && pwd && panorama-cli package-application

### Ready for deploying to a device

Congrats! Your app is now ready to deploy to a device. Next, you can continue in this notebook to deploy the app programmatically or you can go to the Panorama console and deploying using the AWS Console. The console makes it easier to select camera streams and select the devices you want to deploy to. Programmatic deployment is faster to complete and easier to automate.

Let's make sure the device we are deploying to is available.

### Ready for deploying to a device

Congrats! Your app is now ready to deploy to a device. Next, you can continue in this notebook to deploy the app programmatically or you can go to the Panorama console and deploying using the AWS Console. The console makes it easier to select camera streams and select the devices you want to deploy to. Programmatic deployment is faster to complete and easier to automate.

### How to deploy to device?

The [README](README.md) file has detailed instructions on how to deploy. Please go to the deployment section of the README file and follow along the instructions